In [2]:
import pandas as pd
import folium

In [3]:
ph_map_center = [12.8797, 121.7740]
linear = folium.LinearColormap(["green", "yellow", "orange", "red"], vmin=0., vmax=17.)
linear

In [4]:
df = pd.read_pickle("wind/wind_speed_train.pkl")
df

date    lon    lat   windvelo
0      2007-01-01  116.5  21.50  11.752896
1      2007-01-01  116.5  21.25  11.805607
2      2007-01-01  116.5  21.00  11.851288
3      2007-01-01  116.5  20.75  11.921542
4      2007-01-01  116.5  20.50  12.004713
...           ...    ...    ...        ...
421735 2017-12-01  127.5   5.00   2.709202
421736 2017-12-01  127.5   4.75   2.788130
421737 2017-12-01  127.5   4.50   2.789209
421738 2017-12-01  127.5   4.25   2.844410
421739 2017-12-01  127.5   4.00   2.982223

[421740 rows x 4 columns]

In [5]:
def max_bounds_ne(df, grid_size=0.25):
    adjust = grid_size / 2
    top_left = [ df.lat.max() + adjust, df.lon.min() - adjust ]
    bottom_right = [ df.lat.min() - adjust, df.lon.max() + adjust ]
    return [ top_left, bottom_right ]

In [6]:
ph_max_bounds = max_bounds_ne(df)

In [7]:
def get_square_bounds_ne(ctr_lat, ctr_lon, side_len=0.25):
    adjust = side_len / 2
    top_left = [ ctr_lat + adjust, ctr_lon - adjust ]
    top_right = [ ctr_lat + adjust, ctr_lon + adjust ]
    bottom_left = [ ctr_lat - adjust, ctr_lon - adjust ]
    bottom_right = [ ctr_lat - adjust, ctr_lon + adjust ]
    return [ top_left, bottom_right ]

In [8]:
df["bounds"] = df.apply(lambda x : get_square_bounds_ne(x["lat"], x["lon"]), axis=1)

In [9]:
df

date    lon    lat   windvelo  \
0      2007-01-01  116.5  21.50  11.752896   
1      2007-01-01  116.5  21.25  11.805607   
2      2007-01-01  116.5  21.00  11.851288   
3      2007-01-01  116.5  20.75  11.921542   
4      2007-01-01  116.5  20.50  12.004713   
...           ...    ...    ...        ...   
421735 2017-12-01  127.5   5.00   2.709202   
421736 2017-12-01  127.5   4.75   2.788130   
421737 2017-12-01  127.5   4.50   2.789209   
421738 2017-12-01  127.5   4.25   2.844410   
421739 2017-12-01  127.5   4.00   2.982223   

                                        bounds  
0       [[21.625, 116.375], [21.375, 116.625]]  
1       [[21.375, 116.375], [21.125, 116.625]]  
2       [[21.125, 116.375], [20.875, 116.625]]  
3       [[20.875, 116.375], [20.625, 116.625]]  
4       [[20.625, 116.375], [20.375, 116.625]]  
...                                        ...  
421735    [[5.125, 127.375], [4.875, 127.625]]  
421736    [[4.875, 127.375], [4.625, 127.625]]  
421737    [[4.625, 127.375], [4.375, 127.625]]  
421738    [[4.375, 127.375], [4.125, 127.625]]  
421739    [[4.125, 127.375], [3.875, 127.625]]  

[421740 rows x 5 columns]

In [10]:
df.describe()

lon            lat       windvelo
count  421740.000000  421740.000000  421740.000000
mean      122.000000      12.750000       4.311828
std         3.246797       5.123481       2.926960
min       116.500000       4.000000       0.004706
25%       119.250000       8.250000       1.985798
50%       122.000000      12.750000       3.750518
75%       124.750000      17.250000       6.082603
max       127.500000      21.500000      16.687559

In [11]:
df_07 = df[(df["date"] >= "2007-01-01") & (df["date"] < "2007-02-01")]
df_07

date    lon    lat   windvelo  \
0    2007-01-01  116.5  21.50  11.752896   
1    2007-01-01  116.5  21.25  11.805607   
2    2007-01-01  116.5  21.00  11.851288   
3    2007-01-01  116.5  20.75  11.921542   
4    2007-01-01  116.5  20.50  12.004713   
...         ...    ...    ...        ...   
3190 2007-01-01  127.5   5.00   5.186126   
3191 2007-01-01  127.5   4.75   4.856045   
3192 2007-01-01  127.5   4.50   4.510494   
3193 2007-01-01  127.5   4.25   4.214081   
3194 2007-01-01  127.5   4.00   3.951552   

                                      bounds  
0     [[21.625, 116.375], [21.375, 116.625]]  
1     [[21.375, 116.375], [21.125, 116.625]]  
2     [[21.125, 116.375], [20.875, 116.625]]  
3     [[20.875, 116.375], [20.625, 116.625]]  
4     [[20.625, 116.375], [20.375, 116.625]]  
...                                      ...  
3190    [[5.125, 127.375], [4.875, 127.625]]  
3191    [[4.875, 127.375], [4.625, 127.625]]  
3192    [[4.625, 127.375], [4.375, 127.625]]  
3193    [[4.375, 127.375], [4.125, 127.625]]  
3194    [[4.125, 127.375], [3.875, 127.625]]  

[3195 rows x 5 columns]

In [12]:
def get_color_hex(value):
    return linear(value)[:7]

In [13]:
def plot_wind_velo_values(df):
    ph_map = folium.Map(location=ph_map_center, tiles="cartodbpositron", zoom_start=6)
    ph_map.fit_bounds(bounds=ph_max_bounds)
    df.apply(lambda x :
             folium.Rectangle(x["bounds"], stroke=False, color=get_color_hex(x["windvelo"]),
                                 fill=True, fillColor=get_color_hex(x["windvelo"])).add_to(ph_map), axis=1)
    ph_map

In [25]:
df_list = {}
for i in range(0, 421740, 3195):
    df_slice = df.iloc[i : i + 3195].reset_index()
    index = str(df_slice["date"].min())[:10]
    df_list[index] = df_slice

len(df_list)

132

In [27]:
df_list["2007-03-01"]

index       date    lon    lat  windvelo  \
0      6390 2007-03-01  116.5  21.50  5.715897   
1      6391 2007-03-01  116.5  21.25  5.667705   
2      6392 2007-03-01  116.5  21.00  5.628857   
3      6393 2007-03-01  116.5  20.75  5.625746   
4      6394 2007-03-01  116.5  20.50  5.638504   
...     ...        ...    ...    ...       ...   
3190   9580 2007-03-01  127.5   5.00  6.587872   
3191   9581 2007-03-01  127.5   4.75  6.445368   
3192   9582 2007-03-01  127.5   4.50  6.271881   
3193   9583 2007-03-01  127.5   4.25  6.061558   
3194   9584 2007-03-01  127.5   4.00  5.834051   

                                      bounds  
0     [[21.625, 116.375], [21.375, 116.625]]  
1     [[21.375, 116.375], [21.125, 116.625]]  
2     [[21.125, 116.375], [20.875, 116.625]]  
3     [[20.875, 116.375], [20.625, 116.625]]  
4     [[20.625, 116.375], [20.375, 116.625]]  
...                                      ...  
3190    [[5.125, 127.375], [4.875, 127.625]]  
3191    [[4.875, 127.375], [4.625, 127.625]]  
3192    [[4.625, 127.375], [4.375, 127.625]]  
3193    [[4.375, 127.375], [4.125, 127.625]]  
3194    [[4.125, 127.375], [3.875, 127.625]]  

[3195 rows x 6 columns]